In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# test
# Load the CSV data into a DataFrame
df = pd.read_csv('eth_transactions.csv')

# Display the first 5 rows of the DataFrame
print(df.head())

   Unnamed: 0                                               hash  nonce  \
0           0  0xa611438e5637c227e5080477b7180fc3d1c76710f2aa...     14   
1           1  0xd5d2fe97f1fa4b772476e208f1e9a3441a0f54a020ac...      3   
2           2  0xc93e15158aa51a4d9a93c3c153868c8c89f4545c3445...     19   
3           3  0x454bc286bda9cf62a43730e465df8e76c23124d6dde7...     17   
4           4  0xba053083f0752cd5f9a152105698dba490d5992b9839...    993   

   transaction_index                                from_address  \
0                 41  0x506a48155c891c78c04bc0b70eb1d9b2361635e0   
1                 59  0x544c7de319b72b557a0c1fafccf5202813f9f3f7   
2                 76  0x1562c07b34b828cedaadda6959a0b1362103b7b4   
3                 83  0x0afaec415159835aab6d56fbc6a22e1d60c76c45   
4                 90  0x7bd8477901552199e8208f62875aa94fab2fffb3   

                                   to_address         value     gas  \
0  0x7cc46cf6392ed9db45351a3f82eb2cc3b83da761  0.000000e+00  157911  

## EDA (Exploratory Data Analysis)

In [2]:
print(df.shape)
print(df.columns)

# About our Data

<p>
Unnamed: 0: This is likely an index column from the original dataset. It doesn't carry any meaningful information for our analysis.

hash: This is the unique identifier of the transaction.

nonce: This is a value that can only be used once. It's used to prevent the same transaction from being processed more than once.

transaction_index: This is the position of the transaction in the block.

from_address: This is the address of the sender of the transaction.

to_address: This is the address of the receiver of the transaction.

value: This is the amount of Ether being transferred in the transaction.

gas: This is the amount of "gas" provided for the transaction. Gas in Ethereum is the measure of computational effort.

gas_price: This is the price of gas in Gwei (1 Gwei = 1e-9 Ether) set by the sender of the transaction.

input: This is an optional data field that can be included in a transaction. It's used when the transaction is sent to a smart contract.

receipt_cumulative_gas_used: This is the total amount of gas used in the block when this transaction was processed.

receipt_gas_used: This is the amount of gas used by this particular transaction.

receipt_contract_address: If the transaction was a contract creation, this is the address of the created contract.

receipt_status: This is the status of the transaction - '1' means the transaction was successful, and '0' means it failed.

block_hash: This is the unique identifier of the block that includes this transaction.

block_number: This is the number of the block that includes this transaction.

block_timestamp: This is the timestamp of when the block was mined.
</p>

In [3]:
print(df.isnull().sum())

In [4]:
print(df.describe())

In [5]:
corr_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

Strong Positive Correlation:
Values close to +1 imply a strong positive correlation. This means that as one variable increases, the other variable also increases. For example, a correlation of +0.8 or higher is typically considered strong.

Strong Negative Correlation:
Values close to -1 indicate a strong negative correlation. This means that as one variable increases, the other decreases. Similarly, a correlation of -0.8 or lower is often considered strong in the negative direction.
No Correlation:

A value close to 0 implies no correlation. This means there is no linear relationship between the two variables.


## Data Cleaning Process

In [6]:
# Drop unnecessary columns
columns_to_drop = ['Unnamed: 0','receipt_root']
df = df.drop(columns=columns_to_drop)

In [7]:
# Handle missing values - Since the 'to_address' column has missing values, we fill them with a placeholder 
# (Ethereum transactions to a null address are typically contract creation transactions)
df['to_address'] = df['to_address'].fillna('ContractCreation')

In [8]:
# handle receipt_contract_address missing values
df['receipt_contract_address'] = df['receipt_contract_address'].fillna('Unknown address')

In [9]:
# handle max_priority_fee_per_gas missing values
df['max_fee_per_gas'].fillna(df['max_fee_per_gas'].mean(), inplace=True)
df['max_priority_fee_per_gas'].fillna(df['max_priority_fee_per_gas'].mean(), inplace=True)

In [10]:
print(df.isnull().sum())

In [11]:
df.head()

In [12]:
# Convert the 'block_timestamp' to datetime
df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])

### Transaction Flow Analysis: 

Using from_address and to_address, we can analyze transaction flows, identifying common senders and receivers, and understanding the network of transactions.

In [13]:
# Number of unique addresses
unique_from_addresses = df['from_address'].nunique()
unique_to_addresses = df['to_address'].nunique()

print("unique_from_addresses",unique_from_addresses)
print("unique_to_addresses",unique_to_addresses)

# which addreses appear to sent more than others

In [14]:
print(df['from_address'].value_counts().head(10))

# Top 10 popular addresses to receive

In [15]:
print(df['to_address'].value_counts().head(10))

In [16]:
# Graph
# import networkx as nx


# top_5_senders = df['from_address'].value_counts().head(5).index
# top_5_receivers = df['to_address'].value_counts().head(5).index

# # Filtering the data to include only transactions involving these top addresses
# simplified_data = df[(df['from_address'].isin(top_5_senders)) | (df['to_address'].isin(top_5_receivers))]

# # Creating a simplified network graph
# G_simplified = nx.DiGraph()

# for _, row in simplified_data.iterrows():
#     G_simplified.add_edge(row['from_address'], row['to_address'], weight=row['value'])

# plt.figure(figsize=(10, 10))
# pos_simplified = nx.spring_layout(G_simplified, seed=42)  # for consistent layout
# nx.draw_networkx(G_simplified, pos_simplified, node_size=700, node_color='lightblue', alpha=0.8, arrows=True)
# plt.title('Simplified Transaction Flow Among Top Ethereum Addresses')
# plt.show()

# When?

## Temporal Patterns: 

Here, we'll plot the number of transactions over time, looking for hourly or minute patterns.

In [17]:
# Temporal Patterns: Analyzing transaction counts by hour

# Extracting hour from the timestamp
df['hour'] = df['block_timestamp'].dt.hour

# Grouping data by hour to count transactions
transaction_counts_by_hour = df.groupby('hour').size()

# Plotting the number of transactions per hour
plt.figure(figsize=(12, 6))
transaction_counts_by_hour.plot(kind='bar')
plt.title('Number of Transactions Per Hour')
plt.xlabel('Hour of Day (0-23)')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=0)
plt.grid(True)
plt.show()

In [18]:
# Temporal Patterns: Analyzing transaction counts by minute
from matplotlib.ticker import MaxNLocator

# Extracting hour from the timestamp
# Extracting minute and hour from the timestamp for more granular analysis
df['minute_of_hour'] = df['block_timestamp'].dt.minute
df['hour_of_day'] = df['block_timestamp'].dt.hour

# Combining date, hour, and minute for a unique time identifier
df['date_hour_minute'] =  df['hour_of_day'].astype(str) + ':' + df['minute_of_hour'].astype(str)


# Grouping data by hour to count transactions
transaction_counts_by_hour = df.groupby('date_hour_minute').size()

# Plotting the number of transactions per hour
plt.figure(figsize=(12, 6))
transaction_counts_by_hour.plot(kind='bar')
plt.title('Number of Transactions Per minute')
plt.xlabel('Minute of Hour (0-60)')
plt.ylabel('Number of Transactions')

# Customizing the x-axis labels
ax = plt.gca()  # Get the current Axes instance
ax.xaxis.set_major_locator(MaxNLocator(nbins=30)) # Adjust 'nbins' as needed


plt.xticks(rotation=90)
plt.grid(True)
plt.show()

## Gas and Transaction Efficiency: 

We can visualize the distribution of gas used and gas prices, as well as how they change over time.

In [19]:
# Distribution of Gas Used
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['gas'], bins=50, kde=False)
plt.title('Distribution of Gas Used in Transactions')
plt.xlabel('Gas Used')
plt.ylabel('Frequency')
plt.yscale('log')  # Log scale due to wide range of values

By visualizing the distribution of gas used in transactions, we can identify the typical range of gas consumption. This can help us understand the average complexity of transactions on the Ethereum network. Unusually high gas consumption may indicate complex smart contract interactions or inefficient transaction practices.

In [20]:
# Visualizing how Gas Used and Gas Prices change over time
# For simplicity, we will use the average values per hour
df['hour'] = df['block_timestamp'].dt.hour
avg_gas_by_day = df.groupby('hour')['gas'].mean()
avg_gas_price_by_day = df.groupby('hour')['gas_price'].mean()

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
avg_gas_by_day.plot(kind='line')
plt.title('Average Gas Used Per Transaction Over Time By Hour')
plt.xlabel('Hour')
plt.ylabel('Average Gas Used')

plt.subplot(1, 2, 2)
avg_gas_price_by_day.plot(kind='line')
plt.title('Average Gas Price Over Time By Hour')
plt.xlabel('Hour')
plt.ylabel('Average Gas Price (Gwei)')

plt.tight_layout()
plt.show()

We can identify periods of congestion or low activity based on fluctuations in gas prices. High gas prices may indicate network congestion and increased demand for transaction processing, while low gas prices may suggest periods of low activity.

Correlation between Gas Price and Gas Used:

Analyzing the relationship between gas prices and gas used in transactions can help us understand user behavior and network dynamics. For example, we might observe that users are willing to pay higher gas prices for transactions with larger gas consumption, indicating a willingness to prioritize more complex or high-value transactions during periods of congestion.

In [21]:
# For simplicity, we will use the average values per minute

# Extracting hour from the timestamp
# Extracting minute and hour from the timestamp for more granular analysis
df['minute_of_hour'] = df['block_timestamp'].dt.minute
df['hour_of_day'] = df['block_timestamp'].dt.hour

# Combining date, hour, and minute for a unique time identifier
df['date_hour_minute'] =  df['hour_of_day'].astype(str) + ':' + df['minute_of_hour'].astype(str)



avg_gas_by_day = df.groupby('date_hour_minute')['gas'].mean()
avg_gas_price_by_day = df.groupby('date_hour_minute')['gas_price'].mean()

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
avg_gas_by_day.plot(kind='line')
plt.title('Average Gas Used Per Transaction Over Time By Minute')
plt.xlabel('Minute')
plt.ylabel('Average Gas Used')

plt.subplot(1, 2, 2)
avg_gas_price_by_day.plot(kind='line')
plt.title('Average Gas Price Over Time By Minute')
plt.xlabel('Minute')
plt.ylabel('Average Gas Price (Gwei)')

plt.tight_layout()
plt.show()

Average Gas Used Per Transaction by Minute:

</br>

This line chart shows the average gas used in transactions for each minute within an hour. While minute-to-minute variations might be more subtle, this chart can still highlight any short-term fluctuations in gas usage, potentially revealing finer-grained patterns or anomalies.

Average Gas Price by Minute:

</br>

This chart illustrates the average gas price (in Gwei) for each minute within an hour. Like the gas usage, minute-level variations might be less pronounced but can still provide insights into very short-term pricing dynamics.

## Smart Contract Interactions: 

</br>
We'll visualize the proportion of transactions that interact with smart contracts.

In [22]:
# Smart Contract Interactions: Visualizing the proportion of transactions interacting with smart contracts

# Identifying transactions with non-empty 'input' field as smart contract interactions
df['is_smart_contract_interaction'] = df['input'].apply(lambda x: x != '0x')

# Calculating the proportion of transactions that are smart contract interactions
smart_contract_interaction_proportion = df['is_smart_contract_interaction'].mean()

# Visualizing the proportion
plt.figure(figsize=(8, 6))
plt.pie([smart_contract_interaction_proportion, 1 - smart_contract_interaction_proportion], 
        labels=['Smart Contract Interactions', 'Other Transactions'], 
        autopct='%1.1f%%', startangle=140, colors=['skyblue', 'lightgrey'])
plt.title('Proportion of Transactions Interacting with Smart Contracts')
plt.show()

Insights:

Usage of Smart Contracts: The chart provides a clear view of how prevalent smart contract interactions are in the Ethereum network. A larger segment for smart contract interactions would indicate a significant use of Ethereum for more than just Ether transfers, highlighting its role as a platform for decentralized applications.

Network Activity Composition: Understanding the proportion of smart contract interactions can give insights into the composition of network activity, which is crucial for both users and developers. It helps in gauging the network's complexity and the demand for computational resources.

## Contract Creation Activity: 

We can plot the number of new contracts created over time.

In [23]:
# Contract Creation Activity: Plotting the number of new contracts created by minute
df['is_contract_creation'] = df['receipt_contract_address'].notna()

# Extracting minute and hour from the timestamp for more granular analysis
df['minute_of_hour'] = df['block_timestamp'].dt.minute
df['hour_of_day'] = df['block_timestamp'].dt.hour

# Combining date, hour, and minute for a unique time identifier
df['date_hour_minute'] =  df['hour_of_day'].astype(str) + ':' + df['minute_of_hour'].astype(str)

# Grouping data by this unique time identifier to count new contract creations
contract_creation_counts_minute = df.groupby('date_hour_minute')['is_contract_creation'].sum()

# Plotting the number of new contracts created by minute
# Note: This will be a large plot, so we'll limit it to a subset of the data
sampled_contract_creation_counts_minute = contract_creation_counts_minute.head(1000)

plt.figure(figsize=(15, 6))
sampled_contract_creation_counts_minute.plot(kind='line')
plt.title('Number of New Contracts Created Over Time (By Minute)')
plt.xlabel('Time (Date Hour:Minute)')
plt.ylabel('Number of New Contracts Created')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


## Insights:
Granular View of Contract Creation: Plotting the data by minute offers a granular view of contract creation activity. This can reveal short-term spikes or patterns that might be related to specific events or periods of high activity.

High-Frequency Fluctuations: Unlike daily or hourly aggregations, minute-level data can exhibit more fluctuations, reflecting the immediacy of activities on the network.

Potential for Anomaly Detection: This level of detail can be useful for detecting anomalies or unusual bursts of activity, which might be indicative of specific events or trends in the developer community.

Limitations of Minute-Level Analysis: It's important to note that such fine-grained analysis may sometimes yield noisy data, making it challenging to discern clear trends. Additionally, the plot covers only a small portion of the entire dataset due to its size and complexity.

## Transaction Success and Failures: 

The receipt_status column is crucial for understanding the reliability of the network. Analyzing the proportion of successful vs. failed transactions, and identifying patterns or common reasons for failures can be insightful.

In [24]:
# Transaction Success and Failures: Visualizing the proportion of successful and failed transactions
# Calculating the proportion of transaction statuses
transaction_success_proportion = df['receipt_status'].value_counts(normalize=True)

# Visualizing the proportion
plt.figure(figsize=(8, 6))
transaction_success_proportion.plot(kind='bar', color=['green', 'red'])
plt.title('Proportion of Transaction Success vs. Failures')
plt.xlabel('Status (1=Success, 0=Failure)')
plt.ylabel('Proportion')
plt.xticks(rotation=0)
plt.grid(True)
plt.show()

## Insights:
Network Reliability: The chart provides a clear indication of the Ethereum network's reliability. A high proportion of successful transactions suggests a stable and reliable network.

Impact of Network Conditions: The rate of failed transactions can be influenced by network conditions. For example, during times of high congestion or rapid changes in gas prices, the rate of failures might increase.

User Experience: The ratio of successes to failures can also reflect the user experience. A lower rate of failures implies that users are generally able to execute their transactions without issues.

Optimization Opportunities: For developers and network operators, understanding the proportion of failed transactions can help in identifying areas for optimization and improvement within the network or in user interfaces.

## Block-Level Analysis: 

We'll visualize the number of transactions per block and other related block-level metrics.

In [25]:
# Block-Level Analysis: Visualizing the number of transactions per block and other related block-level metrics

# Counting the number of transactions per block
transactions_per_block = df.groupby('block_number').size()

# Visualizing the number of transactions per block
plt.figure(figsize=(12, 6))
transactions_per_block.plot(kind='line')
plt.title('Number of Transactions Per Block')
plt.xlabel('Block Number')
plt.ylabel('Number of Transactions')
plt.grid(True)
plt.show()

# Calculating additional block-level metrics such as average gas used and average gas price per block
avg_gas_per_block = df.groupby('block_number')['gas'].mean()
avg_gas_price_per_block = df.groupby('block_number')['gas_price'].mean()

# Visualizing these metrics
plt.figure(figsize=(15, 6))

# Average Gas Used Per Block
plt.subplot(1, 2, 1)
avg_gas_per_block.plot(kind='line')
plt.title('Average Gas Used Per Block')
plt.xlabel('Block Number')
plt.ylabel('Average Gas Used')

# Average Gas Price Per Block
plt.subplot(1, 2, 2)
avg_gas_price_per_block.plot(kind='line')
plt.title('Average Gas Price Per Block (Gwei)')
plt.xlabel('Block Number')
plt.ylabel('Average Gas Price (Gwei)')

plt.tight_layout()
plt.show()


### Insights:
Block Utilization Trends: Understanding the number of transactions per block is crucial for assessing the Ethereum network's capacity and efficiency. It also helps in gauging the scalability of the network.

Resource Usage Patterns: The average gas used per block reflects the network's resource usage patterns. Blocks consistently using high amounts of gas might indicate a need for optimization or scaling solutions.

Gas Price Dynamics: The average gas price per block can provide insights into the economic aspects of the Ethereum network, particularly how users respond to changes in network demand.

# Which addresses have the highest average

In [26]:
# Average transaction value per sender address
average_value_per_sender = df.groupby('from_address')['value'].mean()

# Top 10 sender addresses with the highest average transaction value
top_value_senders = average_value_per_sender.sort_values(ascending=False).head(10)

(top_value_senders)

In [27]:
# Average transaction value per receiver address
average_value_per_receiver = df.groupby('to_address')['value'].mean()
# Top 10 receiver addresses with the highest average transaction value
top_value_receivers = average_value_per_receiver.sort_values(ascending=False).head(10)
(top_value_receivers)

In [28]:
# Descriptive statistics for the 'value' column
value_description = df['value'].describe()

# Distribution of the transaction 'value'
plt.figure(figsize=(10, 6))
plt.hist(df['value'], bins=30, log=True, color='skyblue', edgecolor='black')
plt.title('Distribution of Transaction Values')
plt.xlabel('Value')
plt.ylabel('Frequency (log scale)')
plt.grid(True)
plt.show()

(value_description)

# For reference ethereum price was 3,510.33$ at the time 

So 5.36 * 3,510.33 is approximately 18.815$

In [29]:
df.head()

<br/>
<br/>

### The distribution of transaction values is highly skewed, with most transactions having low values. This is common in financial data, where a large number of transactions are for small amounts, and a small number of transactions are for large amounts.

<br/>
<br/>


# Feature Engineering

hour_of_day: This represents the hour of the day (in 24-hour format) when the block containing the transaction was added to the blockchain.

day_of_week: This represents the day of the week (Monday=0, Sunday=6) when the block containing the transaction was added to the blockchain.

month: This represents the months when the block containing the transaction was added to the blockchain.
    
year: This represents the hour of the year when the block containing the transaction was added to the blockchain.
    
total_transactions_sender: This represents the total number of transactions made by the sender address in the dataset.

total_transactions_receiver: This represents the total number of transactions received by the receiver address in the dataset.

average_value_sender: This represents the average value of transactions made by the sender address in the dataset.

average_value_receiver: This represents the average value of transactions received by the receiver address in the dataset.

In [30]:
# Create time-based features
df['hour_of_day'] = df['block_timestamp'].dt.hour
df['minute'] = df['block_timestamp'].dt.minute
df['day_of_week'] = df['block_timestamp'].dt.dayofweek
df['month'] = df['block_timestamp'].dt.month
df['year'] = df['block_timestamp'].dt.year

# Total transactions made by each sender address
total_transactions_sender = df.groupby('from_address').size()
df['total_transactions_sender'] = df['from_address'].map(total_transactions_sender)

# Total transactions received by each receiver address
total_transactions_receiver = df.groupby('to_address').size()
df['total_transactions_receiver'] = df['to_address'].map(total_transactions_receiver)

# Average value of transactions made by each sender address
average_value_sender = df.groupby('from_address')['value'].mean()
df['average_value_sender'] = df['from_address'].map(average_value_sender)

# Average value of transactions received by each receiver address
average_value_receiver = df.groupby('to_address')['value'].mean()
df['average_value_receiver'] = df['to_address'].map(average_value_receiver)



df.head()

### From Address Metrics ( Aggregation):

Average Transaction Value: The mean value of Ether transferred in transactions for each address on each day.

Average Gas Used: The mean amount of gas used in transactions for each address on each day.

Average Gas Price: The mean price of gas (in Gwei) set by transactions for each address on each day.

Total Transactions: The total number of transactions sent from each address on each day.

Total Value Sent: The total amount of Ether sent from each address on each day.

In [31]:
# Aggregating by day for 'from_address'
address_hour_agg = df.groupby(['from_address', 'date_hour_minute']).agg(
    average_transaction_value=('value', 'mean'),
    average_gas_used=('gas', 'mean'),
    average_gas_price=('gas_price', 'mean'),
    total_transactions=('hash', 'count'),
    total_value_sent=('value', 'sum')
)

# Displaying the first few rows of the aggregated data
address_hour_agg.head(10)

In [32]:
# Placeholder addresses for demonstration
selected_addresses = ['0x21a31ee1afc51d94c2efccaa2092ad1028285549' ]  
# Replace with actual addresses

plt.figure(figsize=(15, 6))

for address in selected_addresses:
    # Filtering the data for the selected address
    address_data = address_hour_agg.loc[address]

    # Plotting average transaction value over time
    plt.plot(address_data.index, address_data['average_transaction_value'], label=address)

plt.xlabel('Date Hour Minute')
plt.ylabel('Average Transaction Value')
plt.title('Average Transaction Value Over Time for Selected Addresses')
plt.legend()
plt.grid(True)

# Customizing the x-axis labels
ax = plt.gca()  # Get the current Axes instance
ax.xaxis.set_major_locator(MaxNLocator(nbins=30)) # Adjust 'nbins' as needed


plt.xticks(rotation=45)
plt.show()


In [33]:
plt.figure(figsize=(15, 6))

for address in selected_addresses:
    # Filtering the data for the selected address
    address_data = address_hour_agg.loc[address]

    # Plotting average transaction value over time
    plt.plot(address_data.index, address_data['average_gas_used'], label=address)

plt.xlabel('Date Hour Minute')
plt.ylabel('Average Gas Used')
plt.title('Average Gas Used Over Time for Selected Addresses')
plt.legend()
plt.grid(True)

# Customizing the x-axis labels
ax = plt.gca()  # Get the current Axes instance
ax.xaxis.set_major_locator(MaxNLocator(nbins=30)) # Adjust 'nbins' as needed


plt.xticks(rotation=45)
plt.show()

In [34]:
plt.figure(figsize=(15, 6))

for address in selected_addresses:
    # Filtering the data for the selected address
    address_data = address_hour_agg.loc[address]

    # Plotting average transaction value over time
    plt.plot(address_data.index, address_data['average_gas_price'], label=address)

plt.xlabel('Date Hour Minute')
plt.ylabel('Average Gas Price')
plt.title('Average Gas Price Over Time for Selected Addresses')
plt.legend()
plt.grid(True)

# Customizing the x-axis labels
ax = plt.gca()  # Get the current Axes instance
ax.xaxis.set_major_locator(MaxNLocator(nbins=30)) # Adjust 'nbins' as needed


plt.xticks(rotation=45)
plt.show()


In [35]:
plt.figure(figsize=(15, 6))

for address in selected_addresses:
    # Filtering the data for the selected address
    address_data = address_hour_agg.loc[address]

    # Plotting average transaction value over time
    plt.plot(address_data.index, address_data['total_transactions'], label=address)

plt.xlabel('Date Hour Minute')
plt.ylabel('Total Transactions')
plt.title('Total Transactions Over Time for Selected Addresses')
plt.legend()
plt.grid(True)

# Customizing the x-axis labels
ax = plt.gca()  # Get the current Axes instance
ax.xaxis.set_major_locator(MaxNLocator(nbins=30)) # Adjust 'nbins' as needed


plt.xticks(rotation=45)
plt.show()


Time series plots of transaction metrics can provide several insights useful for anomaly detection, especially when observing metrics like average transaction value, gas used, and gas prices over time. Here are some key insights you might derive:

Sudden Spikes or Drops: Sharp increases or decreases in transaction values, gas used, or gas prices can indicate anomalous activity. For instance, a sudden spike in transaction value might suggest large transfers that are not typical for the address.

Outliers: Points that deviate significantly from the general pattern could be potential outliers. These might be transactions that are unusually high or low compared to the normal activity of the address.

Pattern Changes: Anomalies might be indicated by changes in the usual patterns of activity. For example, if an address consistently shows low transaction values and suddenly starts transacting in high values, it could be considered anomalous.

Periodicity and Trends: Regular patterns like cyclical activity or trends (upward or downward) can be normal for some addresses. A break from these patterns could signal something unusual.

Consistency with Known Events: If there are known market events or network changes (like Ethereum upgrades), see if the anomalies align with these events. Anomalies that coincide with such events might be explainable and not necessarily indicative of suspicious activity.

Comparison Across Addresses: If you're observing multiple addresses, anomalies might also be identified by comparing their activities. An address behaving distinctly from others in a similar category could be flagged for further investigation.

## Pick an algorithm:

- we will go with Isolation Forest or K-means or DBSCAN, ideally all

## Training

# K-MEANS

In [36]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
import numpy as np

# Assuming 'df' is your DataFrame and the features are selected
features = df[['hour_of_day', 'day_of_week', 'month', 'year', 'total_transactions_sender', 'total_transactions_receiver', 'average_value_sender', 'average_value_receiver']]

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features.fillna(0))  # Handling nulls by replacing them with 0

# Applying K-means
kmeans = KMeans(n_clusters=5)  # Example with 5 clusters
kmeans.fit(X_scaled)
labels = kmeans.labels_

# Finding the distances of each point to its cluster center
distances = np.min(cdist(X_scaled, kmeans.cluster_centers_, 'euclidean'), axis=1)

# Marking the points which are significantly far away from the cluster centers as anomalies
# You can define a threshold based on your understanding of the data
threshold = np.percentile(distances, 95)  # Setting threshold as the 95th percentile for example
anomalies = distances > threshold

# Adding the anomaly labels to your DataFrame
df['anomaly'] = anomalies


# Isolation Forest

In [37]:
from sklearn.ensemble import IsolationForest

# Isolation Forest
# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Applying Isolation Forest
# 5% of data as anomalies
iso_forest = IsolationForest(n_estimators=100, contamination=0.05)  
iso_forest.fit(X_scaled)

# Predicting anomalies (1 for normal, -1 for anomaly)
anomalies_iso = iso_forest.predict(X_scaled)

# Adding the anomaly labels to the DataFrame (True for anomalies, False for normal)
df['anomaly_iso'] = anomalies_iso == -1

df.head()  

# DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

# Assuming 'df' is your DataFrame and the features are selected
features = df[['hour_of_day', 'day_of_week', 'month', 'year', 'total_transactions_sender', 'total_transactions_receiver', 'average_value_sender', 'average_value_receiver']]

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features.fillna(0))  # Handling nulls by replacing them with 0

# Applying DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)  
dbscan.fit(X_scaled)

# Extracting the cluster labels (-1 indicates outliers/anomalies)
labels_dbscan = dbscan.labels_

# Marking the outliers/anomalies
anomalies_dbscan = labels_dbscan == -1

# Adding the anomaly labels to your DataFrame
df['anomaly_dbscan'] = anomalies_dbscan

df.head()

# Visualisation

In [ ]:
# Visualizing the clusters
plt.figure(figsize=(10, 6))
plt.scatter(features['average_value_sender'], features['total_transactions_sender'], c=labels_dbscan, cmap='viridis', s=50, alpha=0.5)
plt.title('DBSCAN Clustering')
plt.xlabel('Average Value Sender')
plt.ylabel('Total Transactions Sender')
plt.colorbar(label='Cluster Label')
plt.show()

# Visualizing the anomalies
plt.figure(figsize=(10, 6))
plt.scatter(features['average_value_sender'], features['total_transactions_sender'], c=anomalies_dbscan, cmap='coolwarm', s=50, alpha=0.5)
plt.title('Anomalies Detected by DBSCAN')
plt.xlabel('Average Value Sender')
plt.ylabel('Total Transactions Sender')
plt.colorbar(label='Anomaly')
plt.show()


n this example, the anomalies are defined as the points which are in the top 5% of distances from the cluster center. The choice of the threshold and the number of clusters (k) should be tailored to your specific dataset and the nature of the anomalies you expect to find.

# Analyzing All Algotithm's Results
# Columns where all agree are true

In [ ]:
allAlgorithmsAgreeAnomaly = df[(df['anomaly'] == True) & (df['anomaly_iso'] == True) & (df['anomaly_dbscan'] == True)][["anomaly_iso","anomaly","anomaly_dbscan"]]

In [ ]:
allAlgorithmsAgreeAnomaly

# Columns where at least on of them is true

In [ ]:
atLeastOneisAnomaly = df[(df['anomaly'] == True) | (df['anomaly_iso'] == True) | (df['anomaly_dbscan'] == True)][["anomaly_iso","anomaly","anomaly_dbscan"]]

# Columns where all agree that are normal

In [ ]:
df[(df['anomaly'] == False) & (df['anomaly_iso'] == False) & (df['anomaly_dbscan'] == False)][["anomaly_iso","anomaly","anomaly_dbscan"]]

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))

# Plotting non-anomalous data
sns.scatterplot(data=df[(df['anomaly'] == False) & 
                        (df['anomaly_iso'] == False) & 
                        (df['anomaly_dbscan'] == False)], 
                x='average_value_sender', 
                y='total_transactions_sender', 
                color='blue', 
                label='Normal')

# Plotting anomalous data detected by all algorithms
sns.scatterplot(data=df[(df['anomaly'] == True) & 
                        (df['anomaly_iso'] == True) & 
                        (df['anomaly_dbscan'] == True)], 
                x='average_value_sender', 
                y='total_transactions_sender', 
                color='red', 
                label='Anomaly (All Algorithms Agree)')

# Anomalies where algorithms disagree
# Plotting anomalies detected by any algorithm
# sns.scatterplot(data=df[(df['anomaly'] != True) & 
#                         (df['anomaly_iso'] != True) & 
#                         (df['anomaly_dbscan'] != True)], 
#                 x='average_value_sender', 
#                 y='total_transactions_sender', 
#                 color='orange', 
#                 label='Anomaly (Disagreement)')

plt.title('Anomalies in Ethereum Transactions')
plt.xlabel('Average Value Sent by Sender')
plt.ylabel('Total Transactions by Sender')
plt.legend()
plt.show()


This visualization will help you quickly identify how anomalies differ in characteristics from normal data points. Remember to adjust the x and y-axis labels to match the features you're most interested in exploring.

# Who are these addreses

In [ ]:
anomalous_transactions = df[(df['anomaly'] == True) & 
                             (df['anomaly_iso'] == True) & 
                             (df['anomaly_dbscan'] == True)]

sender_addresses = anomalous_transactions['from_address'].unique()
receiver_addresses = anomalous_transactions['to_address'].unique()

print("Anomalous sender addresses:\n", sender_addresses)
print("\n")
print("Anomalous receiver addresses:\n", receiver_addresses)

# Evaluation of the  Algorithms

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming 'df' is your DataFrame and the features are selected
features = df[['hour_of_day', 'day_of_week', 'month', 'year', 'total_transactions_sender', 'total_transactions_receiver', 'average_value_sender', 'average_value_receiver']]

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features.fillna(0))  # Handling nulls by replacing them with 0
y = df['anomaly']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels_dbscan_test = dbscan.fit_predict(X_test)
anomalies_dbscan_test = labels_dbscan_test == -1

# Isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination=0.05)
anomalies_iso_test = iso_forest.fit_predict(X_test) == -1

# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
anomalies_knn_test = knn.predict(X_test)

# Evaluation
print("DBSCAN Classification Report:")
print(classification_report(y_test, anomalies_dbscan_test))

print("Isolation Forest Classification Report:")
print(classification_report(y_test, anomalies_iso_test))

print("KNN Classification Report:")
print(classification_report(y_test, anomalies_knn_test))


DBSCAN:

DBSCAN achieves a high precision for anomaly detection, meaning that when it flags a transaction as anomalous, it's usually correct. However, it has a relatively low recall, indicating that it misses many true anomalies.
The overall accuracy and f1-score are decent, but the low recall suggests that DBSCAN may not be capturing all anomalies effectively.

Isolation Forest:

Isolation Forest shows improvement over DBSCAN in terms of recall, capturing a higher proportion of true anomalies. However, its precision is moderate, indicating that some of the transactions flagged as anomalies may be false positives.
The overall accuracy and f1-score are comparable to DBSCAN, but there's room for improvement, especially in precision and recall.

KNN:

KNN performs exceptionally well in all aspects, achieving high precision, recall, accuracy, and f1-score for anomaly detection.
It shows almost perfect performance with very few false positives and false negatives.

Given these results, KNN appears to be the best-performing algorithm for this particular anomaly detection task. It provides the highest precision, recall, and overall accuracy among the three algorithms evaluated. If computational efficiency and interpretability are not significant concerns, KNN would be the preferred choice for anomaly detection in this scenario.

# Let's visualise the best algorithms results

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))

# Plotting non-anomalous data
sns.scatterplot(data=df[(df['anomaly'] == False)], 
                x='average_value_sender', 
                y='total_transactions_sender', 
                color='blue', 
                label='Normal')

# Plotting anomalous data detected by all algorithms
sns.scatterplot(data=df[(df['anomaly'] == True)], 
                x='average_value_sender', 
                y='total_transactions_sender', 
                color='red', 
                label='Anomaly (KNN only)')

plt.title('Anomalies in Ethereum Transactions')
plt.xlabel('Average Value Sent by Sender')
plt.ylabel('Total Transactions by Sender')
plt.legend()
plt.show()


# We can check addreses on etherscan

In [ ]:
print("Anomalous receiver addresses:\n", receiver_addresses)

In [ ]:
# Saving Data
# df.to_csv("./eth_transactions_clustered.csv", index=False)